# Step 2

## 2.1 Present the following information for each stock in your equity portfolio:  Market-Cap, Market Beta, 12-month Volatility. 

In [1]:
def convert_market_cap_string_to_numeric(market_cap_string):
    # Dictionary to map suffixes to their numeric values
    suffixes = {'K': 1e3, 'M': 1e6, 'B': 1e9, 'T': 1e12}
    
    # Extract numeric part and suffix from the string
    numeric_part, suffix = market_cap_string[:-1], market_cap_string[-1]
    
    # Convert the numeric part to a float and apply the suffix multiplier
    numeric_value = float(numeric_part) * suffixes.get(suffix.upper(), 1)
    
    return numeric_value

In [4]:
import pandas as pd
from yahoo_fin import stock_info as si
import yfinance as yf
import numpy as np
import dataframe_image as dfi

# Define the ticker symbol and market index symbol
stock_ticker = ['MCHP', 'CIEN', 'WDC', 'INTC', 'KLAC', 'SANM', 'MU', 
                'ADBE', 'MA', 'CRM', 'IPGP', 'LRCX', 'HPQ', 'VNT', 'PYPL']

company_name = ['MICROCHIP TECHNOLOGY INC',
 'CIENA CORP',
 'WESTERN DIGITAL CORP',
 'INTEL CORP',
 'K L A CORP',
 'SANMINA CORP',
 'MICRON TECHNOLOGY INC',
 'ADOBE INC',
 'MASTERCARD INC',
 'SALESFORCE INC',
 'I P G PHOTONICS CORP',
 'LAM RESH CORP',
 'H P INC',
 'VONTIER CORP',
 'PAYPAL HOLDINGS INC']

market_index_ticker = "^GSPC"  # S&P 500

ticker = []
market_cap = []
beta = [] 
volatility = []

# Get historical stock prices for the stock and the market index
for stock in stock_ticker:
    stock_daily_prices = yf.Ticker(stock).history(start="2022-12-30", end="2023-11-12", interval='1d')["Close"]
    stock_price_2022 = stock_daily_prices.iloc[0]
    last_price = stock_daily_prices.iloc[-1]
    
    current_market_cap = yf.Ticker(stock).fast_info.get('marketCap',float('nan'))  
    
    # Convert market cap to billion
    marketcap = (current_market_cap*stock_price_2022)/(last_price*1000000000)
   
    #market bata calculation using monthly data
    stock_prices = yf.Ticker(stock).history(start="2017-12-31", end="2022-12-31", interval='1mo')
    market_prices = yf.Ticker(market_index_ticker).history(start="2017-12-31", end="2022-12-31", interval='1mo')

    # Calculate monthly returns
    stock_returns = stock_prices['Close'].pct_change().dropna()
    market_returns = market_prices['Close'].pct_change().dropna()

    # Calculate covariance between stock and market
    covariance = stock_returns.cov(market_returns)

    # Calculate variance of the market
    variance = market_returns.var()

   # Get historical data for the past 12 months
    hist = yf.Ticker(stock).history(start="2021-12-31", end="2022-12-31", interval='1d')["Close"].pct_change().dropna()
    Volatility = hist.std() * (252 ** 0.5)  # Annualize the volatility
    # Beta is the covariance divided by the variance
    ticker.append(stock)
    market_cap.append(np.round(marketcap,3))
    beta.append(np.round((covariance / variance),2))
    volatility.append(np.round(Volatility,2))

data = pd.DataFrame()
data["Ticker"] = ticker
data["Company Name"] = company_name
data["Market Cap ($B)"] = market_cap
data["Market Cap ($B)"] = data["Market Cap ($B)"].apply(lambda x: float("{:.3f}".format(x))).astype(str)
data["Beta"] = beta
data["Beta"] = data["Beta"].apply(lambda x: float("{:.2f}".format(x))).astype(str)
data["Volatility"] = volatility
data["Volatility"] = data["Volatility"].apply(lambda x: float("{:.2f}".format(x))).astype(str)
data = data.set_index('Ticker')

headers = {
    "selector": "th:not(.index_name)",
    "props": "background-color: whitesmoke; color: black;"
}

index_names = {
    "selector": ".index_name",
    "props": "background-color: whitesmoke; color: black;"
}

data = data.style.set_properties(**{'background-color': 'whitesmoke', 'color': 'black', 'font-size': '110%'}).set_table_styles([index_names, headers])

dfi.export(data,"stockreturns.jpg")

data

,Company Name,Market Cap ($B),Beta,Volatility
Ticker,,,,
MCHP,MICROCHIP TECHNOLOGY INC,40.922,1.58,0.47
CIEN,CIENA CORP,7.68,0.94,0.44
WDC,WESTERN DIGITAL CORP,10.764,1.55,0.48
INTC,INTEL CORP,125.363,0.75,0.38
KLAC,K L A CORP,53.009,1.37,0.49
SANM,SANMINA CORP,3.424,1.16,0.35
MU,MICRON TECHNOLOGY INC,56.936,1.3,0.47
ADBE,ADOBE INC,157.194,1.22,0.45
MA,MASTERCARD INC,329.749,1.08,0.32


## Step 2.2- ETFs: 3 Bond ETFs and 2 Leveraged ETFs

In [3]:
#Select ETFS: Bond ETFS are VUSB, VCIT, VCEB; Leveraged ETFs are TQQQ and SOXL
etf_ticker = ['BND', 'BNDX', 'VCIT','TQQQ','SOXL']

#Download Beta, Expense ratio, and dividend yied for each ETF
etf = []
div_yield = []
expense_ratio = []
Beta = []
returns = []

for ticker in etf_ticker:
    beta = si.get_stats(ticker).loc[5]["Value"]
    divyield = si.get_stats(ticker).loc[3]["Value"]
    expenseratio = si.get_stats(ticker).loc[6]["Value"]
    etf.append(ticker)
    div_yield.append(divyield)
    Beta.append(beta)
    expense_ratio.append(expenseratio)

# Plot all the information    
data = pd.DataFrame()

data["ETFs"] = etf
data["Names"] = ["Vanguard Total Bond Market Index Fund ETF", "Vanguard Total International Bond Index Fund ETF",
                  "Vanguard Intermediate-Term Corp Bond Idx Fund ETF", "ProShares UltraPro QQQ",
                 "Direxion Daily Semiconductor Bull 3X Shares"]
data["Beta"] = Beta
data["Beta"] = Beta
data["Expense Ratio"] = expense_ratio
data["Dividend Yield"] = div_yield

headers = {
    "selector": "th:not(.index_name)",
    "props": "background-color: #001632; color: white;"
}

index_names = {
    "selector": ".index_name",
    "props": "background-color: #001632; color: white;"
}

data = data.set_index('ETFs')

data = data.style.set_properties(**{'background-color': '#001632', 'color': 'white', 'font-size': '110%'}).set_table_styles([index_names, headers])

data

,Names,Beta,Expense Ratio,Dividend Yield
ETFs,,,,
BND,Vanguard Total Bond Market Index Fund ETF,1.00,0.03%,3.15%
BNDX,Vanguard Total International Bond Index Fund ETF,0.74,0.11%,2.06%
VCIT,Vanguard Intermediate-Term Corp Bond Idx Fund ETF,1.22,0.07%,3.85%
TQQQ,ProShares UltraPro QQQ,3.41,0.88%,1.57%
SOXL,Direxion Daily Semiconductor Bull 3X Shares,4.29,0.94%,0.90%
